## In this python code, we will create Multiple Linear Regression Models for both the datasets

In [1]:
import csv
import itertools
import pickle
import pandas as pd
import numpy as np
from scipy import stats
import time
import statsmodels.api as sm

start_time = time.time()

In [2]:
pd.set_option('display.max_columns', None)

#### This function converts p-values in stars, lesser the p-values, more the stars
It is done to better visualize the statistical significance of the results.

In [3]:
def starTeller(_value):
    if _value == 0:
        return '*****'
    if _value > 0 and _value <=0.001:
        return '****'
    if _value > 0.001 and _value <=0.01:
        return '***'
    if _value > 0.01 and _value <=0.05:
        return '**'
    if _value > 0.05 and _value <=0.1:
        return '*'
    if _value > 0.1:
        return '.'

#### Choosing the dataset

In [4]:
datasetNo = 2
if datasetNo == 1:
    dataset = pickle.load(open('dataOCM/dataset_1_for_regression.data', 'rb'))
    topicFeaturesList = ['topic' + str(i) for i in range(18)]
elif datasetNo == 2:
    dataset = pickle.load(open('dataOCM/dataset_2_for_regression.data', 'rb'))
    topicFeaturesList = ['topic' + str(i) for i in range(23) if i not in [12,20,22]]
    # print(dataset.head(3))

In [5]:
dataset.head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic21,Year,Recommendation,One_Hot_Position_Employee,One_Hot_Position_Management,One_Hot_Position_Qualification,One_Hot_Position_TemporaryEmployed,PreviousVsCurrentFlag,RvScoreScaled,RvScoreWorkAtmosphereScaled,RvScoreCohesionAmongColleaguesScaled,RvScoreEqualRightsScaled,RvScoreDealingWithOlderColleaguesScaled,RvScoreEnvironmentalSocialAwarenessScaled,RvScoreWorkLifeBalanceScaled,Sentiment Score Normalized
1,0.000000,0.071954,0.000000,0.000000,0.000000,0.00000,0.000000,0.227733,0.071895,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.112077,0.137255,0.000000,0.071894,0.000000,1.00,1.0,1.0,0.0,0.0,0.0,1.0,0.90,1.0,1.0,0.94,1.0,0.94,0.8,0.672652
2,0.000000,0.000000,0.111966,0.000000,0.000000,0.00000,0.079119,0.000000,0.076924,0.086509,0.279106,0.000000,0.000000,0.076640,0.000000,0.000000,0.000000,0.000000,0.177839,0.000000,0.91,0.0,1.0,0.0,0.0,0.0,0.0,0.64,0.6,0.6,0.60,0.8,0.40,0.6,0.514455
3,0.010753,0.010753,0.010756,0.225807,0.010753,0.11828,0.010753,0.010753,0.010754,0.010755,0.440842,0.010753,0.010754,0.010753,0.010753,0.010756,0.010754,0.010753,0.010753,0.010753,0.91,1.0,0.0,1.0,0.0,0.0,1.0,0.96,1.0,1.0,1.00,1.0,1.00,0.8,0.506261


In [6]:
dataset.describe()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic21,Year,Recommendation,One_Hot_Position_Employee,One_Hot_Position_Management,One_Hot_Position_Qualification,One_Hot_Position_TemporaryEmployed,PreviousVsCurrentFlag,RvScoreScaled,RvScoreWorkAtmosphereScaled,RvScoreCohesionAmongColleaguesScaled,RvScoreEqualRightsScaled,RvScoreDealingWithOlderColleaguesScaled,RvScoreEnvironmentalSocialAwarenessScaled,RvScoreWorkLifeBalanceScaled,Sentiment Score Normalized
count,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.00000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7058.000000,7.058000e+03
mean,0.040716,0.032604,0.056716,0.023585,0.041290,0.028808,0.038876,0.030913,0.036477,0.045948,0.101802,0.038335,0.054970,0.027431,0.037486,0.035041,0.048021,0.030303,0.044165,0.023614,0.770153,0.566095,0.753896,0.135591,0.079343,0.03117,0.735761,0.677359,0.668963,0.756767,0.710646,0.724437,0.679489,0.675792,5.203493e-01
std,0.080341,0.059130,0.098059,0.046433,0.079459,0.056781,0.069976,0.071477,0.064525,0.086248,0.141541,0.063914,0.105309,0.050892,0.070268,0.069625,0.105585,0.058458,0.075504,0.042814,0.226131,0.458481,0.430770,0.342378,0.270292,0.17379,0.440958,0.230207,0.287169,0.247746,0.285874,0.281168,0.271107,0.280150,7.048119e-02
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,4.829994e-09
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.640000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.480000,0.400000,0.600000,0.420000,0.600000,0.400000,0.400000,4.864370e-01
50%,0.013699,0.013699,0.015873,0.012048,0.013699,0.012048,0.013699,0.012048,0.013699,0.013700,0.018868,0.013699,0.013700,0.012049,0.013699,0.012051,0.012048,0.012052,0.013700,0.012048,0.820000,0.500000,1.000000,0.000000,0.000000,0.00000,1.000000,0.720000,0.800000,0.800000,0.800000,0.800000,0.800000,0.800000,5.217768e-01
75%,0.023256,0.023256,0.030304,0.018868,0.023256,0.023256,0.023257,0.018868,0.030303,0.030303,0.186943,0.030303,0.030303,0.023256,0.023256,0.023256,0.023256,0.023256,0.030303,0.023256,0.910000,1.000000,1.000000,0.000000,0.000000,0.00000,1.000000,0.880000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,5.585053e-01
max,0.698610,0.493969,0.716777,0.492057,0.805280,0.493974,0.650753,0.650780,0.561633,0.650773,0.763427,0.495050,0.856203,0.488363,0.493971,0.650778,0.698618,0.584876,0.584890,0.584905,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000e+00


#### listing dependent and independent variables

In [7]:
independents = topicFeaturesList + ['featureYear',
                                    'featureRecom',
                                    'featureSentiScorenormalized',
                                    "PreviousVsCurrentFlag",
                                    "One_Hot_Position_Employee",
                                    "One_Hot_Position_Management",
                                    "One_Hot_Position_Qualification",
                                    "One_Hot_Position_TemporaryEmployed"]
dependentsUnscaled = ['RvScore',
              'RvScoreWorkAtmosphere',
              'RvScoreCohesionAmongColleagues',
              'RvScoreEqualRights',
              'RvScoreDealingWithOlderColleagues',
              'RvScoreEnvironmentalSocialAwareness',
              'RvScoreWorkLifeBalance']

all_features = independents + dependentsUnscaled
all_features

['topic0',
 'topic1',
 'topic2',
 'topic3',
 'topic4',
 'topic5',
 'topic6',
 'topic7',
 'topic8',
 'topic9',
 'topic10',
 'topic11',
 'topic13',
 'topic14',
 'topic15',
 'topic16',
 'topic17',
 'topic18',
 'topic19',
 'topic21',
 'featureYear',
 'featureRecom',
 'featureSentiScorenormalized',
 'PreviousVsCurrentFlag',
 'One_Hot_Position_Employee',
 'One_Hot_Position_Management',
 'One_Hot_Position_Qualification',
 'One_Hot_Position_TemporaryEmployed',
 'RvScore',
 'RvScoreWorkAtmosphere',
 'RvScoreCohesionAmongColleagues',
 'RvScoreEqualRights',
 'RvScoreDealingWithOlderColleagues',
 'RvScoreEnvironmentalSocialAwareness',
 'RvScoreWorkLifeBalance']

#### splitting the dataset into training and test data: Train = 80%, Test = 20%

In [8]:
# with random masking
mask = np.random.rand(len(dataset)) < .8
datasetTrain = dataset[mask]
datasetTest = dataset[~mask]

In [9]:
datasetTrain.describe()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic21,Year,Recommendation,One_Hot_Position_Employee,One_Hot_Position_Management,One_Hot_Position_Qualification,One_Hot_Position_TemporaryEmployed,PreviousVsCurrentFlag,RvScoreScaled,RvScoreWorkAtmosphereScaled,RvScoreCohesionAmongColleaguesScaled,RvScoreEqualRightsScaled,RvScoreDealingWithOlderColleaguesScaled,RvScoreEnvironmentalSocialAwarenessScaled,RvScoreWorkLifeBalanceScaled,Sentiment Score Normalized
count,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5634.000000,5.634000e+03
mean,0.040866,0.032732,0.056244,0.023094,0.041403,0.029647,0.038929,0.030881,0.036070,0.046369,0.103075,0.038856,0.054885,0.027542,0.037097,0.034738,0.048092,0.029822,0.044037,0.023992,0.771626,0.564608,0.752751,0.137735,0.077742,0.031771,0.732517,0.676400,0.667889,0.755424,0.710123,0.722928,0.677089,0.674826,5.197668e-01
std,0.080241,0.059143,0.097720,0.045638,0.080043,0.058101,0.069627,0.070836,0.063343,0.086472,0.142552,0.064692,0.105421,0.050915,0.069303,0.068760,0.105574,0.057403,0.074922,0.043842,0.225139,0.459139,0.431451,0.344652,0.267789,0.175406,0.442686,0.230295,0.287418,0.248454,0.286663,0.281643,0.271884,0.279805,7.055895e-02
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,4.829994e-09
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.640000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.480000,0.400000,0.600000,0.400000,0.600000,0.400000,0.400000,4.862547e-01
50%,0.013699,0.013699,0.015873,0.012048,0.013699,0.012048,0.013699,0.012048,0.013699,0.013700,0.018868,0.013699,0.013700,0.012049,0.013699,0.012052,0.012049,0.012051,0.013700,0.012048,0.820000,0.500000,1.000000,0.000000,0.000000,0.000000,1.000000,0.720000,0.800000,0.800000,0.800000,0.800000,0.800000,0.800000,5.213494e-01
75%,0.023256,0.023256,0.030304,0.018868,0.023256,0.023256,0.023257,0.018868,0.030303,0.030303,0.189311,0.030303,0.023265,0.023256,0.023256,0.023256,0.023256,0.023256,0.030303,0.023256,0.910000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.880000,1.000000,1.000000,1.000000,1.000000,0.992000,1.000000,5.576416e-01
max,0.698610,0.492054,0.716777,0.492057,0.805280,0.493974,0.650753,0.584903,0.561633,0.650773,0.763427,0.495050,0.856203,0.398060,0.493971,0.650778,0.698618,0.584876,0.584890,0.584905,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.916171e-01


In [10]:
datasetTest.describe()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic13,topic14,topic15,topic16,topic17,topic18,topic19,topic21,Year,Recommendation,One_Hot_Position_Employee,One_Hot_Position_Management,One_Hot_Position_Qualification,One_Hot_Position_TemporaryEmployed,PreviousVsCurrentFlag,RvScoreScaled,RvScoreWorkAtmosphereScaled,RvScoreCohesionAmongColleaguesScaled,RvScoreEqualRightsScaled,RvScoreDealingWithOlderColleaguesScaled,RvScoreEnvironmentalSocialAwarenessScaled,RvScoreWorkLifeBalanceScaled,Sentiment Score Normalized
count,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000,1424.000000
mean,0.040123,0.032099,0.058580,0.025528,0.040842,0.025487,0.038666,0.031041,0.038087,0.044284,0.096768,0.036274,0.055308,0.026992,0.039029,0.036243,0.047743,0.032206,0.044674,0.022119,0.764326,0.571980,0.758427,0.127107,0.085674,0.028792,0.748596,0.681152,0.673213,0.762082,0.712718,0.730405,0.688985,0.679613,0.522654
std,0.080759,0.059097,0.099400,0.049423,0.077135,0.051106,0.071364,0.073983,0.069004,0.085362,0.137404,0.060715,0.104898,0.050819,0.073970,0.072956,0.105666,0.062443,0.077790,0.038457,0.230003,0.455984,0.428187,0.333210,0.279981,0.167281,0.433973,0.229903,0.286242,0.244941,0.282823,0.279302,0.267892,0.281578,0.070150
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.200000,0.167380
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.640000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.480000,0.400000,0.600000,0.549500,0.600000,0.537000,0.400000,0.486972
50%,0.013699,0.013699,0.015873,0.012048,0.013699,0.012048,0.013699,0.012048,0.013699,0.013700,0.018868,0.013699,0.013701,0.012050,0.013699,0.012050,0.012048,0.013699,0.013703,0.012048,0.820000,0.500000,1.000000,0.000000,0.000000,0.000000,1.000000,0.740000,0.800000,0.800000,0.800000,0.800000,0.800000,0.800000,0.522685
75%,0.023256,0.023256,0.067184,0.022895,0.023256,0.018868,0.023256,0.018868,0.030303,0.023261,0.173937,0.030303,0.030303,0.023256,0.023257,0.023256,0.023256,0.023256,0.030303,0.018869,0.910000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.880000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.561273
max,0.614439,0.493969,0.698615,0.396220,0.584873,0.493974,0.414631,0.650780,0.488332,0.584870,0.763417,0.430462,0.650767,0.488363,0.492063,0.488372,0.650794,0.495290,0.584884,0.333335,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Running Regression in all the possible combinations of the variables, providing that topics should be present in all the combinations

In [11]:
# stuff = independents
stuff =['Year',
        'Recommendation',
        'Sentiment Score Normalized',
        "PreviousVsCurrentFlag",
        'oneHots',
        'topics']

oneHots = ["One_Hot_Position_Employee",
            "One_Hot_Position_Management",
            "One_Hot_Position_Qualification",
            "One_Hot_Position_TemporaryEmployed"]

# fetching all the combinations
independentsCombos = []
for L in range(1, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        # print(list(subset))
        independentsCombos.append(list(subset))

for k in range(len(independentsCombos)):
    if 'topics' in independentsCombos[k]:
        independentsCombos[k].remove('topics')
        independentsCombos[k].extend(topicFeaturesList)
    if 'oneHots' in independentsCombos[k]:
        independentsCombos[k].remove('oneHots')
        independentsCombos[k].extend(oneHots)

        
stuff.remove('topics')
stuff.extend(topicFeaturesList)
stuff.remove('oneHots')
stuff.extend(oneHots)

#### looping though all the models and reporting R2 values for all

In [12]:
results = []
# Dependents -->> Dependent Variables:: From Pearson's correlation code
dependents = ['RvScoreScaled', 'RvScoreWorkAtmosphereScaled', 'RvScoreCohesionAmongColleaguesScaled', 'RvScoreEqualRightsScaled', 'RvScoreDealingWithOlderColleaguesScaled', 'RvScoreEnvironmentalSocialAwarenessScaled', 'RvScoreWorkLifeBalanceScaled']
values = [0 for i in range(len(stuff))] # Check

for m in range(len(dependents)):
    currentY = dependents[m]
    dfObj = pd.DataFrame(columns=['independent variables', 'Model Number', 'R2 on training set', 'MSE training set', 'R2 on test set', 'MSE on test set', 'Intercept', 'NoOfFeatures', 'NoOfSignificantFeatures'] + stuff)

    for i in range(len(independentsCombos)):
        if 'topic0' not in independentsCombos[i]:
            continue
        resultsTemp = []
        train_x = np.asanyarray(datasetTrain[independentsCombos[i]])
        train_y = np.asanyarray(datasetTrain[[currentY]])
        test_x = np.asanyarray(datasetTest[independentsCombos[i]])
        test_y = np.asanyarray(datasetTest[[currentY]])
        
        train_y_flat= train_y.flatten()
        test_y_flat = test_y.flatten()

        train_x_2 = sm.add_constant(train_x)  # adding a constant
        test_x = sm.add_constant(test_x)
        ols_model = sm.OLS(train_y, train_x_2).fit()
        predictions_train = ols_model.predict(train_x_2)
        predictions_test = ols_model.predict(test_x)        

        ### code to calculate RMSE
        sq_error_ = np.square(predictions_train - train_y_flat)
        mse = np.mean(sq_error_)
        
        sq_error_test = np.square(predictions_test - test_y_flat)
        mse_test = np.mean(sq_error_test)
        
        rSqTrain = ols_model.rsquared
        #test_x_2 = sm.add_constant(test_x)
        rSqTest = (sm.OLS(test_y, test_x).fit()).rsquared

        dictCoeffs = dict(zip(stuff, values))
        dictPees=dict(zip(stuff, values))
        dictPeesStar = dict(zip(stuff, values))
        dicTees=dict(zip(stuff, values))
        
        for a in range(len(independentsCombos[i])):
            dictCoeffs[independentsCombos[i][a]] = ols_model.params[a+1]#lasso_reg2.coef_[a]
            dictPees[independentsCombos[i][a]] = ols_model.pvalues[a+1]#list(pvalues)[a]
            dictPeesStar[independentsCombos[i][a]] = starTeller(dictPees[independentsCombos[i][a]])
            dicTees[independentsCombos[i][a]] = ols_model.tvalues[a+1]#list(tvalues)[a]

        # Pass the row elements as key value pairs to append() function
        # Dictionary and list are created to save the result in excel sheet
        dictCoeffs['independent variables'] = independentsCombos[i]
        dictCoeffs['R2 on training set'] = rSqTrain
        dictCoeffs['MSE training set'] = mse
        dictCoeffs['R2 on test set'] = rSqTest
        dictCoeffs['MSE on test set'] = mse_test
        dictCoeffs['NoOfFeatures'] = len(independentsCombos[i])

        dictCoeffs['Model Number'] = i+1
        dictPees['Model Number'] = i+1
        dictPeesStar['Model Number'] = i+1
        dicTees['Model Number'] = i+1

        dictCoeffs['Intercept'] = ols_model.params[0]  # lasso_reg2.intercept_[0]
        dictPees['Intercept'] = ols_model.pvalues[0]
        dictPeesStar['Intercept'] = starTeller(dictPees['Intercept'])
        dicTees['Intercept'] = ols_model.tvalues[0]

        dictPeesStar['R2 on test set'] = "ProbStar"
        dictPees['R2 on test set'] = "Probs"
        dicTees['R2 on test set'] = "Tees"
        dictPeesStar['NoOfSignificantFeatures'] = sum([1 for jk in ols_model.pvalues if jk <= 0.05])

        dfObj = dfObj.append(dictCoeffs, ignore_index=True)
        dfObj = dfObj.append(dictPeesStar, ignore_index=True)
        dfObj = dfObj.append(dictPees, ignore_index=True)
        dfObj = dfObj.append(dicTees, ignore_index=True)

        #print("*************************************************************************************************************************************************************")
        #print('Processed model number ' + str(i + 1) + ' / ' + str(len(independentsCombos)) + ' of dep var no. ' + str(m+1) + ' / ' + str(len(dependents)) + ' ::: trainR2: ' + str(rSqTrain) + ' testR2: ' + str(rSqTest))
    dfObj.to_excel('dataOCM/RQ2_DATASET'+str(datasetNo)+'_'+ currentY + '.xlsx')
    print('RQ2_DATASET'+str(datasetNo)+'_'+ currentY + '.xlsx created...')
print("--- %s seconds ---" % (time.time() - start_time))

RQ2_DATASET2_RvScoreScaled.xlsx created...
RQ2_DATASET2_RvScoreWorkAtmosphereScaled.xlsx created...
RQ2_DATASET2_RvScoreCohesionAmongColleaguesScaled.xlsx created...
RQ2_DATASET2_RvScoreEqualRightsScaled.xlsx created...
RQ2_DATASET2_RvScoreDealingWithOlderColleaguesScaled.xlsx created...
RQ2_DATASET2_RvScoreEnvironmentalSocialAwarenessScaled.xlsx created...
RQ2_DATASET2_RvScoreWorkLifeBalanceScaled.xlsx created...
--- 23.814826250076294 seconds ---
